# **OCR PDF using TESSERACT AND PADDLEOCR**

In [10]:
from pdf2image import convert_from_path
import pytesseract
import os
import requests
import zipfile
import io
from paddleocr import PaddleOCR
from dotenv import load_dotenv
from utils import (is_cn_block,
                   ocr_layout,
                   extract_vn_letters,
                   extract_cn_letters,)

## **Prepare Environment**

### Download Tessdata for **Tesseract**

In [ ]:
def download_tessdata(lang):
    local_dir = os.path.join(os.getcwd(), 'tessdata')
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)
        
    file_path = os.path.join(local_dir, f'{lang}.traineddata')
    
    # Tải về nếu file chưa tồn tại
    if not os.path.exists(file_path):
        url = f"https://github.com/tesseract-ocr/tessdata/blob/main/{lang}.traineddata"
        response = requests.get(url)
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print(f"Download tessdata for {lang} successfully")
    else:
        print(f"Tessdata for {lang} existed")
        
    return local_dir

In [ ]:
tessdata_path = download_tessdata('vie')
tessdata_path = download_tessdata('chi-sim')

# Khi gọi tesseract để biết sẽ config folder testdata của mình tạo
custom_config = f'--testdata-dir "{tessdata_path}"'

Tessdata for vie existed
Tessdata for chi-sim existed


### Download Poppler for **pdf2image**

In [15]:
def install_poppler():
    local_dir = os.path.join(os.getcwd(), 'Poppler')
    bin_path = os.path.join(local_dir,'poppler-25.11.0', 'Library', 'bin')
    
    url = "https://github.com/oschwartz10612/poppler-windows/releases/download/v25.11.0-0/Release-25.11.0-0.zip"
    
    response = requests.get(url)
    response.raise_for_status()
    
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)
        
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(local_dir)
        
    return bin_path

In [17]:
poppler_path = install_poppler()

## **Run OCR**

In [ ]:
# Config path
load_dotenv()
poppler_path_windows = poppler_path
tesseract_path = os.getenv('TESSERACT_PATH')
pytesseract.pytesseract.tesseract_cmd = tesseract_path

In [ ]:
start_page = 107
end_page = 157

pages = convert_from_path('data/raw/pdf1.pdf', 
                          poppler_path=poppler_path_windows, 
                          dpi=400,
                          first_page = start_page,
                          last_page = end_page,
                          )

In [21]:
paddle_ocr = PaddleOCR(lang='ch', 
                use_doc_orientation_classify=False,
                use_doc_unwarping=False,
                use_textline_orientation=False,)

d:\DOCUMENT\School\NLP\school\Project\mt-project\.venv\Lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\vnviv\.paddlex\official_models\PP-OCRv5_server_det`.
Creating model: ('PP-OCRv5_server_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\vnviv\.paddlex\official_models\PP-OCRv5_server_rec`.


In [22]:
full_text = {}
full_text['CN'] = []
full_text['VN'] = []

for i, page_image in enumerate(pages):
    cn_text, vn_text = ocr_layout(page_image, paddle_ocr, i)
    full_text['CN'].append(cn_text)
    
    # vn_text sẽ chứa tiếng việt và tiếng pinyin -> sẽ xử lý sau
    full_text['VN'].append(vn_text)

In [23]:
letters_vn = []
for v in full_text['VN']:
    letters_vn.extend(extract_vn_letters(v))
    
letters_vn

['Bức thư viết cho chính mình số 251 Độc thân cũng tốt, yêu đương cũng được. Tất cả chỉ là một giai đoạn nào đó trong cuộc đời bạn. Không phải hâm mộ những người xung quanh thì cũng đừng bất mãn với hiện tại. Cho dù có ai đó bên cạnh hay không, phía trước có lời hứa cùng phấn đấu vì một mục tiêu nào đó hay không, mọi cung bậc cảm xúc trong cuộc sống đều cần chính bản thân trải nghiệm qua và đúc kết. Sống đúng thì mỗi giai đoạn đều sẽ trở thành một bản ngã tốt hơn của chính bạn. Trưởng thành không chỉ đơn giản là để thoát khỏi cảnh độc thân mà là khiến viết cho chính mình có thế trải nghiệm tất cả những việc gì “có khả năng”.',
 'Bức thư viết cho chính mình số 252 Làm việc khi tính táo, đọc sách khi mơ hồ, ngủ khi tức giận và suy nghĩ khi ở một mình. Hãy làm một người hạnh phúc, đọc sách, du lịch, nỗ lực làm việc, quan tâm đến cơ thể và tâm lí, đồng thời trở thành người tốt nhất so với chính mình.',
 'Bức thư viết cho chính mình số 253 Con người có ba lần đánh dấu sự trưởng thành: Lần đ

In [24]:
len(letters_vn)

4

In [25]:
letters_cn = []

for v in full_text['CN']:
    letters_cn.extend(extract_cn_letters(v))
    
letters_cn

['写给自己的第251封信 单身也好， 没必要羡慕旁人， 都只是人生中的某个阶段。 恋爱也罢， 无论身边是否有人陪伴， 前方是否有承诺一起努力的 也别对现状抱不满。 正确地活着， 生活中的喜怒哀乐都需要自己去经历并沉淀的。 每个 目标， 阶段都会成就更好的自己。 而是让自己有力量 成长不是单纯地为了脱单， 去体验任何可能。',
 '写给自己的第252封信 清醒时做事， 大怒时睡觉， 独处时思考； 做一个幸福的 糊涂时读书， 成为最好的自己。 关心身体和心情， 努力工作， 读书， 旅行，',
 '写给自己的第253封信 第二次是 第一次是在发现自己不是世界中心的时候。 人会长大三次。 第三次是 终究还是有些事令人无能为力的时候。 在发现即使再怎么努力， 但还是会尽力争取的时候。 在明知道有些事可能会无能为力，',
 '写给自己的第254封信 我学会了勇敢; 我学会了坚强; 怕的时候没人陪， 哭的时候没人哄， 我学会了自 累的时候没人可以依靠， 我学会了承受; 烦的时候没人问， 原来我很优秀， 我 我找到了自己， 更可贵的是， 立……就这样， 世界上， 只有一个我！ 只有一个，',
 '写给自己的第255封信 可以游手好闲而赢得女人的欣赏。 更没有任何一 没有任何一个男人， 而得到一个男人的尊重。 个女人能够好吃懒做， 别 还是埋头干吧， 所以， 没有委屈就没有成长。 把时间浪费在解释上，']

In [26]:
len(letters_cn)

5

In [40]:
import pandas as pd 

df_vn = pd.DataFrame(letters_vn, columns =['vi'])
df_cn = pd.DataFrame(letters_cn[:-1], columns = ['zh'])

In [41]:
df_com = pd.concat([df_vn, df_cn], axis=1, ignore_index=True)

In [ ]:
df_com.to_json("data/processed/output.json")

,0,1
0,Bức thư viết cho chính mình số 251 Độc thân cũ...,写给自己的第251封 信 单身也好， 没必要羡慕旁人， 都只是人生中的某个阶段。 恋爱也罢，...
1,Bức thư viết cho chính mình số 252 Làm việc kh...,写给自己的第252封 信 清醒时做事， 大怒时睡觉， 独处时思考； 做一个幸福的 糊涂时读书...
2,Bức thư viết cho chính mình số 253 Con người c...,写给自己的第253封 信 第二次是 第一次是在发现自己不是世界中心的时候。 人会长大三次。 ...
3,Bức thư viết cho chính mình số 254 Khi khóc kh...,写给自己的第254封 信 我学会了勇敢; 我学会了坚强; 怕的时候没人陪， 哭的时候没人哄，...
4,Bức thư viết cho chính mình số 255 Không có mộ...,写给自己的第255封 信 可以游手好闲而赢得女人的欣赏。 更没有任何一 没有任何一个男人， ...
5,Bức thư viết cho chính mình số 256 Bạn càng ki...,写给自己的第256封 信 你能多快搞定自己的情绪， 就能多快得到成长。脾气会赶走运气。 所 ...
6,Bức thư viết cho chính mình số 257 Bạn có thể ...,写给自己的第257封 信 会是你明天的成 可以心疼， 你可以哭泣， 今天的泪水， 但不能绝望...
7,Bức thư viết cho chính mình số 258 Trên thế gi...,写给自己的第258封 信 当你失去了所有的依靠 这回事， 这个世界上根本就不存在 自然就什么...
8,Bức thư viết cho chính mình số 259 Cuộc sống l...,写给自己的第259封 信 那些受伤的地方一定会变成我 生活总是让我们遍体鳞伤， 但到后来， ...
9,Bức thư viết cho chính mình số 260 Con đường t...,写给自己的第260封 信 如果真的每个人都是鸿鹄的话， 那么便没有燕雀 成长路上无捷径， “...
